In [1]:
import os
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
from matplotlib.pyplot import MultipleLocator
import seaborn as sns
%matplotlib inline

In [2]:
# files_dir = "C:/Users/zhongj/Desktop/Task2_Data/Final_Zonal_Data_for_DA" 
# os.chdir(files_dir)

### General plot

In [3]:
# edit the file 
def edit_df(file):
    df = pd.read_excel(file)
    #add a DateTime Column (format: period):
    df['DateTime'] = pd.PeriodIndex(year = df['Year'], month = df['Month'],
                                    day = df['Day'], hour = df['Hour'], freq = 'H')
    #change format to datatime:
    df['DateTime'] = df['DateTime'].map(lambda x: x.to_timestamp())
    return df

In [4]:
# df = edit_df('ZoneK_FinalData.xlsx')

In [5]:
# def general_plot(df, category, HB, HE):
#     Zone_name = list(df['Zone'].unique())[0] #Zone name
#     n = len(category)
    
#     df2 = df.loc[(df['Hour'] >= HB) &(df['Hour'] <= HE)] #Time period
#     cat_data = df2.loc[:, ['DateTime', category]] # Narrow dataframe
    
#     # plot:
#     fig, ax = plt.subplots(1,1,figsize = (20,8))
#     ax.plot(cat_data['DateTime'], cat_data['%s'%category],'o',markersize=2, alpha = 0.5)
#     # plot setting:
#     ax.set_title('Zone %s - %s Hourly Behavior from %d:00 to %d:00 in 2017-2020'%(Zone_name, 
#                                                                                   category, HB, HE))
#     ax.set_ylabel('%s'%category)
#     ax.grid(color = 'grey', alpha =0.1)
#     ax.xaxis.grid(False)
    
#     plt.show() # show plot in python
#     # save plot in current file dir
#     fig.savefig('Zone%s %s Hourly General Behavior'%(Zone_name, category), bbox_inches="tight")

#     # 3 ways to close plots:
#         # (seems like it doesn't work, but I put them here in case it works)
#     plt.cla()
#     plt.clf()
#     plt.close(fig)

In [6]:
# feature_ = ['DateTime','BTM', 'GHI', 'POA20','POA30']
# general_plot(df, feature_ , 6, 19)

In [51]:
def general_plot(df, features, HB, HE):
    Zone_name = list(df['Zone'].unique())[0] #Zone name
    n = (len(features)-1)
    
    df2 = df.loc[(df['Hour'] >= HB) &(df['Hour'] <= HE)] #Time period
    cat_data = df2[features] # Narrow dataframe
    
    # plot:
    fig, ax = plt.subplots(n,1,figsize = (15,15))
    plt.title('Zone %s - Hourly Behavior from %d:00 to %d:00 in 2017-2020'%(Zone_name, HB, HE))
    
    for i in range(n):
        ax[i].plot(cat_data['DateTime'], cat_data[features[i+1]],'o',markersize=2, alpha = 0.5)
        # plot setting:
        ax[i].set_title('Zone %s - %s - All data points from %d:00 to %d:00 in 2017-2020 (hourly interval)'%(Zone_name, 
                                                                                                        features[i+1], HB, HE))
        ax[i].set_ylabel('%s'%features[i+1])
        ax[i].grid(color = 'grey', alpha =0.1)
        ax[i].xaxis.grid(False)

        # plt.show() # show plot in python
        # save plot in current file dir
    fig.savefig('Zone%s_Solar_Data_Hourly_General_Behavior'%(Zone_name), bbox_inches="tight")

    # 3 ways to close plots:
        # (seems like it doesn't work, but I put them here in case it works)
    plt.cla()
    plt.clf()
    plt.close(fig)

### Contour plot

In [8]:
# input math should be: 'max', 'min', 'sd', 'mean'
def choose_data(df, category, math): 
    Zone_name = list(df['Zone'].unique())[0] # Zone name
    df = df[(df['Hour']>=5)&(df['Hour']<=20)]
    cat_data = df.loc[:, ['Month', 'Hour','%s'%category]] #Choose catergory
   
    
    if math == 'max':
        cat_data = cat_data.groupby(['Month', 'Hour']).max()
        cat_data = cat_data.reset_index()
        math = 'Maximum'
    elif math == 'min':
        cat_data = cat_data.groupby(['Month', 'Hour']).min()
        cat_data = cat_data.reset_index()
        math = 'Minimum Value'
    elif math == 'sd':
        cat_data = cat_data.groupby(['Month', 'Hour']).std()
        cat_data = cat_data.reset_index()
        math = 'Standard Deviation'
    else:
        cat_data = cat_data.groupby(['Month', 'Hour']).mean()
        cat_data = cat_data.reset_index()
        math = 'Average Value'
        
    return cat_data, Zone_name, math

In [9]:
def change_data_structure(cat_data):
    
    x = sorted(list(set(cat_data['Month'].values))) #horizontal axis (column names)
    y = sorted(list(set(cat_data['Hour'].values))) # vertical axis (Row names)
    df_new = cat_data.loc[:, ['Month', 'Hour']]

# code no need to be used:
#     df_new = pd.DataFrame(columns = cat_data.columns.values[:2])  
#     for i in x:
#         curX = np.array([i]*len(y))
#         curX.shape = (len(y),1)
#         curY = np.array(y)
#         curY.shape = (len(y),1)

#         join_h = np.hstack((curX,curY))

#         df_new = df_new.append(pd.DataFrame(join_h, columns = cat_data.columns.values[:2]), 
#                                ignore_index = True)
    
    df_new2 = pd.merge(cat_data, df_new, how='right', on = list(df_new.columns.values))
    
    index_list = []
    for j in y:
        index_list.append(df_new2.loc[df_new2[cat_data.columns.values[1]] == j][cat_data.columns.values[2]].values)
    
    return x, y, index_list

In [32]:
#7 inputs extracted from previous functions
def contour_plot(x, y, index_list, Zone_name, category, math): 
    X,Y = np.meshgrid(x,y) # get x-axis and y-axis
    
    #plot:
    fig, ax = plt.subplots(1,1,figsize = (15,10))
    ax.contourf(X,Y,index_list, 10,cmap = plt.cm.hot)
    c = plt.contour(X,Y, index_list, 10, colors = 'black' )
    
    #setting:
    ax.clabel(c, inline = True, fontsize = 10)
    ax.set_ylabel('Hour')
    ax.set_xlabel('Month')
    ax.set_title('Zone%s - %s of %s of each hour in each month'%(Zone_name, math, category))
    
    # plt.show() # show plot in python
    # save plot in current file dir
    fig.savefig('Zone%s_%s_of_%s_Contour_Plot'%(Zone_name, math, category),bbox_inches="tight")
    
    # 3 ways to close plots:
        # (seems like it doesn't work, but I put them here in case it works)
    plt.cla()
    plt.clf()
    plt.close(fig)

In [11]:
# combine above 3 functions
def contour_plot_final(df, category, math):
    cat_data, Zone_name, math  = choose_data(df, category, math)
    x, y, s = change_data_structure(cat_data)
    contour_plot(x, y, s, Zone_name, category, math)

In [12]:
# convert data to 12*24 dataframe and output a excel file
def data_table(df, category, math):
    cat_data, Zone_name, math  = choose_data(df, category, math) #use previous function
    x, y, s = change_data_structure(cat_data) #use previous function
    data_table  = pd.DataFrame(s,columns = x) # dataframe
    return data_table

### Correlation

In [13]:
# setting for upper triangle of correlation plots:
def corrdot(*args, **kwargs):
    corr_r = args[0].corr(args[1], 'pearson')
    corr_text = f"{corr_r:2.2f}".replace("0.", ".")
    ax = plt.gca()
    ax.set_axis_off()
    marker_size = abs(corr_r) * 10000
    ax.scatter([.5], [.5], marker_size, [corr_r], alpha=0.6, cmap="coolwarm",
               vmin=-1, vmax=1, transform=ax.transAxes)
    font_size = abs(corr_r) * 40 + 5
    ax.annotate(corr_text, [.5, .5,],  xycoords="axes fraction",
                ha='center', va='center', fontsize=font_size)

    sns.set(style='white', font_scale=1.6)

In [14]:
def corr(df, variables, HB, HE):
    #choose data and time range:
    n = len(variables)
    Zone_name = list(df['Zone'].unique())[0] #Zone name
    cor_data = df.loc[:, variables]
    cor_data = cor_data.loc[(df['Hour'] >= HB) &(df['Hour'] <= HE)]
    
    #plot:
    g = sns.PairGrid(cor_data, aspect=1.4, diag_sharey=False)
    g.map_lower(sns.regplot, lowess=True, ci=False, line_kws={'color': 'black'}) #lower triangle
    g.map_diag(sns.distplot, kde_kws={'color': 'black'}) #diagonal
#     g.map_upper(corrdot) #upper triangle
    #title:
    g.fig.suptitle('Zone%s - Correlation Scatter Plots Between Variables'%(Zone_name, HB, HE), y=1.08)
    
    # plt.show() # show plot in python
    # save plot in current file dir
    plt.savefig('Zone%s_Correlation_Scatter_Plots_of_%d_variables'%(Zone_name, n), bbox_inches="tight")
   
    # 2 ways to close plots:
        # (seems like it doesn't work, but I put them here in case it works)
    plt.cla()
    plt.clf()

In [15]:
# variables =  ['BTM', 'GHI', 'POA20','POA30']
# corr(df, variables, 10, 14)

In [16]:
# variables =  ['BTM','_CLC', '_DBT60', '_DEW', '_SSM','_THI','_WCI', '_WSP']
# corr(df, variables, 10, 14)

### Correlation - Heatmap

In [39]:
def corr_hm(df, variables, HB, HE):
    n = len(variables)
   
    Zone_name = list(df['Zone'].unique())[0] #Zone name
    
    #choose data and time range
    cor_data =  df.loc[:, variables]
    cor_data = cor_data.loc[(df['Hour'] >= HB) &(df['Hour'] <= HE)]
    
    #setting:
    plt.figure(figsize=(15,10))
    plt.title('Zone%s - Correlation Between Variables'%(Zone_name))
    
    #plot:
    correlation_mat = cor_data.corr()
    upp_mat = np.triu(correlation_mat)
    sns.heatmap(correlation_mat, annot = True, cmap="RdYlGn_r",mask = upp_mat)

    # plt.show() # show plot in python
    # save plot in current file dir
    plt.savefig('Zone%s_Correlation_Between_%d_Variables'%(Zone_name, n), 
                bbox_inches="tight")
   
    # 2 ways to close plots:
        # (seems like it doesn't work, but I put them here in case it works)
    plt.cla()
    plt.clf()

In [18]:
# variables = ['BTM', 'GHI', 'POA20',
#            'POA30', '_CLC', '_DBT60', '_DEW', '_GHI60', '_SSM', '_THI', 
#              '_WCI','_WDR', '_WET', '_WSP']
# corr_hm(df, variables, 10, 14)

### BTM Correlation Between Zones

In [46]:
def Zonal_corr_plot(df, HB, HE, Zones):   
    df['HourlyInt'] = pd.to_datetime(df['HourlyInt'])
    df['Month'] =  df['HourlyInt'].dt.month
    df['Day'] =  df['HourlyInt'].dt.day
    df['Hour'] =  df['HourlyInt'].dt.hour
    
    #choose data and time range:
    df2 = df.loc[(df['Hour'] >= HB) &(df['Hour'] <= HE)] 
    df2 = df2.groupby(['Year', 'Month', 'Day']).mean()
    df2 = df2.reset_index()
    
    cor_data =  df2.loc[:, Zones]
    
    #setting: 
    plt.figure(figsize=(15,10))
    plt.title('Zonal Cross Correlation')
    
    #plot:
    correlation_mat = cor_data.corr()
    upp_mat = np.triu(correlation_mat)
    sns.heatmap(correlation_mat, annot = True,cmap="RdYlGn_r",mask = upp_mat)

    # plt.show() # show plot in python
    # save plot in current file dir
    plt.savefig('Zonal_Cross_Correlation', bbox_inches="tight")
   
    # 2 ways to close plots:
        # (seems like it doesn't work, but I put them here in case it works)
    plt.cla()
    plt.clf()
    
    return cor_data, HB, HE

In [20]:
# file = 'C:/Users/zhongj/Desktop/Data/Normalized_Solar.xlsx'
# df_zone = pd.read_excel(file)
# Zones = ['A','B','C','D','E','F','G','H','I','J','K','S']
# Zonal_corr_plot(df_zone, 14, 18, Zones)

In [21]:
# output zonal correlation excel file
def data_table2(cor_data): 
    data_table = cor_data.corr()
    return data_table

### Bonus - Correlation between daily Maximum BTM and daily Maximum Temperatures

In [22]:
# amounts: percentage of the data here is 10%; sort_by:here is GHI
    #: top 10% of data sort by GHI value from big to small
def Select_data(df, amounts, sort_by):
    count = round(len(df)* amounts)
    df1 = df.sort_values(by = [sort_by], ascending = False).reset_index(drop = True)
    df1 = df1.iloc[:count]
    df1 = df1.sort_values(by = ['Year','Month', 'Day', 'Hour']).reset_index(drop = True)
    # df1 = df1.groupby(['Year', 'Month', 'Day']).max()
    df1 = df1.loc[(df1['_THI'] >= 70)]
    return df1, amounts

In [23]:
def corr_hm_BTM_Tem(df, variables, amounts, sort_by):
    Zone_name = list(df['Zone'].unique())[0] #Zone name
    
    df1, amounts = Select_data(df, amounts, sort_by)
    
    cor_data = df1.loc[:, variables] #choose data
    
    amounts = amounts*100
    
    # plot:
    g = sns.PairGrid(cor_data, aspect=1.4, diag_sharey=False)
    g.map_lower(sns.regplot, lowess=True, ci=False, line_kws={'color': 'black'})
    g.map_diag(sns.distplot, kde_kws={'color': 'black'})
    g.map_upper(corrdot)
    g.fig.suptitle('Zone%s - Correlation Between BTM, THI(Higher than 70) with top %d%% %s'%(Zone_name, amounts, sort_by), y=1.08)
   
    # plt.show() # show plot in python
    # save plot in current file dir
    plt.savefig('Zone%s_Correlation_Between_BTM_and_Temperatures'%(Zone_name), bbox_inches="tight")
    
    # 2 ways to close plots:
        # (seems like it doesn't work, but I put them here in case it works)
    plt.cla()
    plt.clf()

In [24]:
# variables = ['BTM', '_THI','_DBT60']

# corr_hm_BTM_Tem(df, variables, 0.01, 'GHI')

### Ramping Distributions

In [25]:
import warnings
warnings.filterwarnings('ignore')
#hour_diff: ramping interval

def ramping(df, hour_diff, HB, HE):
    Zone_name = list(df['Zone'].unique())[0] #Zone name
    df1 = df.loc[(df['Hour'] >= HB) &(df['Hour'] <= HE)] #time range
    grouped = df.groupby(['Year', 'Month', 'Day'])
    
    year = list(df['Year'].unique())
    month =list(df['Month'].unique())
    
    df_diff1 = pd.Series()
    
    #get differnce of 'hour_diff' days in each month and each year:
    for i in year:
        for j in month:
            day = list(df[(df['Year'] == i) & (df['Month'] == j)]['Day'].unique())
            for k in day:
                df2 = grouped.get_group((i,j,k))
                df2 = df2.loc[:, 'BTM'].reset_index(drop = True)
                df3 = df2.diff(hour_diff)
                df3 = df3.dropna(axis=0, how='any', inplace=False)
                df_diff1 = pd.concat([df_diff1, df3], ignore_index = True) 
                
    return df_diff1, Zone_name

In [40]:
import scipy.stats as stats
import matplotlib.ticker as mtick

def ramping_plot(df, hour_diff, HB, HE, n_sigma):
    
    df2, Zone_name = ramping(df, hour_diff, HB, HE)
    
    # 3 sigma
    mean = np.mean(df2)
    std = np.std(df2)
    threshhold1 = mean - n_sigma *std
    threshhold2 = mean + n_sigma *std
    
    #plot:
    fig, ax = plt.subplots(1,1,figsize = (15,5))
    n,x,_ = plt.hist(df2, bins= 150, rwidth=0.95, log=True, label = '', alpha = 0.5)
    
    density = stats.gaussian_kde(df2)

    plt.plot(x, density.pdf(x), lw=2)
    
    #setting:
    ax.set_ylabel('Log scaled Frequency')
    ax.set_xlabel('Changes')
    ax.set_title('Zone%s - BTM %dhr ramping behavior from %d:00 to %d:00'%(Zone_name, 
                                                                                  hour_diff, HB, HE))
    ax.grid(axis='y')
    ax.set_facecolor('#d8dcd6')
    xticks = mtick.PercentFormatter(xmax=1, symbol='%', is_latex=False)
    ax.xaxis.set_major_formatter(xticks)
    
    plt.axvline(threshhold1, ls = '--', c = 'r', label = '%dhr %d-sigma'%(hour_diff ,n_sigma))
    plt.axvline(threshhold2, ls = '--', c = 'r')
    plt.legend()

    
    # plt.show() # show plot in python
    # save plot in current file dir
    fig.savefig('Zone%s_BTM_%dhr_ramping_behavior'%(Zone_name, hour_diff), bbox_inches="tight")
    
    # 3 ways to close plots:
        # (seems like it doesn't work, but I put them here in case it works)
    plt.cla()
    plt.clf()
    plt.close(fig)

In [27]:
# ramping_plot(df, 1, 10, 14, 3)

### Deal with all files

Note that when loop through all 12 files, the error "In RendererAgg: Out of memory" might occur. It might be because the loop loads all graphs or datasets into memory, and it runs out of memory in the RAM.

In [50]:
files_dir_old = 'C:/Users/zhongj/Desktop/TASK2_PPT_DATA' # change the directory

files_dir_new = "C:/Users/zhongj/Desktop/TASK2_PPT" # change the directory

from openpyxl.writer.excel import ExcelWriter

In [52]:
#loops through files:
for filename in os.listdir(files_dir_old):
    file_path = os.path.join(files_dir_old, filename)
    df = edit_df(file_path) # read file
    
    # create folder for zones:
    zone_name = filename.split("_")[0]
    if not os.path.exists(files_dir_new + "//" + zone_name):
        os.makedirs(files_dir_new + "//" + zone_name)
        
    output_folder = files_dir_new + "//" + zone_name
    
    os.chdir(output_folder) # in the folder
    
    # For 'BTM', 'GHI', 'POA20','POA30':
    
    feature = ['DateTime','BTM', 'GHI', 'POA20']
    # General plots:
    general_plot(df, feature, 7, 18)
    
    variables =  ['BTM']
    for i in variables:
         # create folders for 'BTM', 'GHI', 'POA20','POA30':
#         if not os.path.exists(output_folder + "//" + i):
#             os.makedirs(output_folder + "//" + i)
            
#         output_folder2 = output_folder + "//" + i
#         os.chdir(output_folder2) # go to the folder
        
    
        # Contour plots:
#         writer = pd.ExcelWriter(r'%s_%s_12_by_24.xlsx'%(zone_name, i), 
#                                 engine = 'xlsxwriter')
        
        maths = ['max']
        for k in maths: 
            contour_plot_final(df, i, k) # Contour plots
#             data_table00 = data_table(df, i, k) #data table
#             data_table00.to_excel(writer, sheet_name = '%s.xlsx'%(k)) # output to excel
            
#         writer.save()
           
    os.chdir(output_folder) #go back to the zone folder
    
    # Correlation plots between 'BTM', 'GHI', 'POA20','POA30'
#     corr(df, variables, 9, 15)
    
#     # Correlation plots between 'BTM' and weather variables
#     variables2 =  ['BTM','_CLC', '_SSM', '_DBT60', '_DEW','_THI','_WCI', '_WSP' ,'_WDR']
#     corr(df, variables2, 9, 15)

    # Correlation heatmap of all variables
    variables3 = ['BTM', 'GHI', 'POA20','POA30', 
                  '_CLC', '_DBT60', '_DEW', '_SSM', '_THI', 
                  '_WCI','_WDR', '_WET', '_WSP']
    corr_hm(df, variables3, 10, 15)

#     # Bonus - Correlation between daily Maximum BTM and daily Maximum Temperatures:
#     variables4 = ['BTM', '_THI', '_DBT60']
#     corr_hm_BTM_Tem(df, variables4, 0.01, 'GHI')

#     # Ramping:
#     hour_diff = [1,3]
#     for i in hour_diff:
#         ramping_plot(df, i, 10, 14, 3)

In [47]:
# Zonal correlation:
os.chdir(files_dir_new)
file = 'C:/Users/zhongj/Desktop/Task2_Data/Normalized_Solar.xlsx'
df_zone = pd.read_excel(file)
Zones = ['A','B','C','D','E','F','G','H','I','J','K','S']

#writer = pd.ExcelWriter('Zonal_Cross_Correlation.xlsx', engine = 'xlsxwriter')

#8am-17pm
cor_data, HB, HE = Zonal_corr_plot(df_zone, 8, 17, Zones)
# df1 = data_table2(cor_data)
# df1.to_excel(writer, sheet_name = 'from %d to %d'%(HB, HE))

# writer.save()

<Figure size 1080x720 with 0 Axes>

In [ ]:
# df1 = df.loc[(df['Hour'] >= 10) &(df['Hour'] <= 14)]
# grouped = df.groupby(['Year', 'Month', 'Day'])

In [ ]:
# # year = list(df['Year'].unique())
# # month =list(df['Month'].unique())
# # df_diff2 = pd.Series([])
# # for i in year:
#     for j in month:
#         day = list(df[(df['Year'] == i) & (df['Month'] == j)]['Day'].unique())
#         for k in day:
#             df2 = grouped.get_group((i,j,k))
#             df2 = df2.loc[:, 'BTM'].reset_index(drop = True)
#             df3 = df2.diff(3)
#             df3 = df3.dropna(axis=0, how='any', inplace=False)
      
#             df4 = df3/df2
#             df4.replace([np.inf, -np.inf], np.nan, inplace = True)
#             df4 = df4.dropna(axis=0, how='any', inplace=False)
#             df_diff2 = pd.concat([df_diff2, df4], ignore_index = True)  

In [ ]:
# from matplotlib import ticker
# fig, ax = plt.subplots(1,1,figsize = (15,5))

# df_diff2.plot.hist(density=True, grid=True, rwidth=0.9, ax=ax)
# df_diff2.plot.kde(ax=ax, legend=False, title='BTM 1 hr ramping behavior :(h2-h1)/h1')
# ax.set_ylabel('Frequency of Occurrence')
# ax.grid(axis='y')
# ax.set_facecolor('#d8dcd6')